In [ ]:
!pip install simpletransformers
!pip install torch
!pip install sentence_transformers

In [2]:
# Gokhan Has - Experiment 03 - B
import os
import ast
import pandas as pd
import numpy as np
import statistics
import scipy

from simpletransformers.ner import NERModel
from operator import itemgetter
from scipy import spatial
from itertools import chain
from sentence_transformers import SentenceTransformer

In [3]:
def read_csv(filepath):
    if os.path.splitext(filepath)[1] != '.csv':
        return  # or whatever
    seps = [';', '\t']  # ',' is default
    encodings = [None, 'utf-8', 'ISO-8859-1', 'utf-16', 'ascii']  # None is default
    for sep in seps:
        for encoding in encodings:
            try:
                return pd.read_csv(filepath, encoding=encoding, sep=sep)
            except Exception:  # should really be more specific
                pass
    raise ValueError("{!r} is has no encoding in {} or seperator in {}"
                     .format(filepath, encodings, seps))

In [35]:
# Install NER model
model_NER = NERModel('bert', '../ner_model/', use_cuda=False)

# Install BERT model by using SentenceTransformer package.
model = SentenceTransformer('dbmdz/bert-base-turkish-128k-cased')

In [39]:
# Read the dataset and initialize dataset values ...
df = read_csv('../datasets/exp2_dataset.csv')

sbert_sentences = []
all_sentences = []
oldVal = 0
newVal = 1
for index in range(0, len(df)):
    newVal = df['SENTENCE_ID'][index]
    if not oldVal == newVal:
        all_sentences.append( {'index': df['SENTENCE_ID'][index], 'sentence': df['SENTENCE'][index]})
        sbert_sentences.append(df['SENTENCE'][index])
        oldVal = newVal
        


print(len(sbert_sentences))
        
#_sentences = sbert_sentences
#sentence_embeddings = model.encode(_sentences)
#np.save("exp3_3926.npy", sentence_embeddings)


3926


In [56]:
# Get the query sentence
query_sentence = "insan öldürmenin cezası nedir"

# Get the query sentence list
query_sentence_list = query_sentence.split(' ')

# Get the query sentence named entities
prediction, model_output = model_NER.predict([query_sentence])
prediction = prediction[0]

print(prediction)

o_element_list = []
ner_query_list = []
for index, tup in enumerate(prediction):
    if not tup[query_sentence_list[index]] == 'O':
        ner_query_list.append(tup)
    else:
        o_element_list.append(query_sentence_list[index])
        
prediction = ner_query_list

prediction


[{'insan': 'B-ELEMENT_OF_CRIME'}, {'öldürmenin': 'I-ELEMENT_OF_CRIME'}, {'cezası': 'I-PUNISHMENT'}, {'nedir': 'O'}]


[{'insan': 'B-ELEMENT_OF_CRIME'},
 {'öldürmenin': 'I-ELEMENT_OF_CRIME'},
 {'cezası': 'I-PUNISHMENT'}]

In [57]:
# Get the query sentence's word's embedding by using ../ner_model. 
query_word_embeddings_list = []

words = []
new_sentences = []
print(query_sentence_list)
for query_index, query_word in enumerate(query_sentence_list):
    if not query_word in o_element_list:
        print(query_word)
        query_word_embeddings = model_output[0][query_index][query_word][0]
        query_word_embeddings_list.append(query_word_embeddings)

['insan', 'öldürmenin', 'cezası', 'nedir']
insan
öldürmenin
cezası


In [58]:
# In this cell, compare dataset's named entity's embeddings and query sentence's word's embeddings...

for i in range(0, len(o_element_list)):
    query_sentence_list.remove(o_element_list[i])


results_list = []

oldIndex = 1
newIndex = 1
controlSentenceId = 1
for k in range(0, len(prediction)):
    control_tag = prediction[k][query_sentence_list[k]]
    result_list = []
    cumle_list = []
    dataset_tag = ""
    for index in range(0, len(df)):
        newIndex = df['SENTENCE_ID'][index] 
        if not (newIndex == oldIndex):
            if len(cumle_list) == 0:
                result_list.append({'sentence_id' : oldIndex, 'distance': 0.0})
            else:
                avg = statistics.mean(cumle_list)
                result_list.append({'sentence_id' : oldIndex, 'distance': avg})
            # result_list.append(cumle_list)
            cumle_list = []
            oldIndex = newIndex
           
            
        dataset_tag = (df['TAG'][index])[2:len(df['TAG'][index])]
        # print(newIndex, df['WORD'][index], dataset_tag)
        if dataset_tag == control_tag[2:len(control_tag)]:
            # convert list string to list
            veriset_emb = ast.literal_eval(df['EMBEDDING'][index])
            # find cosine distance between query word and dataset word tag
            distances = 1 - spatial.distance.cosine(veriset_emb, query_word_embeddings_list[k])
            #cumle_list.append({dataset_tag : distances, 'sentence_id': df['SENTENCE_ID'][index]})
            cumle_list.append(distances)
   
    oldIndex = 1  
    results_list.append(result_list)

    
    
end_results_list = []

for i in range(0, len(results_list)):
    newlist = sorted(results_list[i], key=itemgetter('sentence_id'))
    end_results_list.append(newlist)


if not len(end_results_list) == 0:
    similarity_value_list = [0] * len(end_results_list[0])
    size = len(prediction)

    for i, list_dict in enumerate(end_results_list):
        for index, one_dict in enumerate(list_dict):   
            dict_values = list(one_dict.values())
            similarity_value_list[index] = similarity_value_list[index] + dict_values[1]

    similarity_value_list = [number / size for number in similarity_value_list]
    #print(similarity_value_list)


    results_dict = []
    for i, list_dict in enumerate(end_results_list):
        for index, one_dict in enumerate(list_dict):
            dict_values = list(one_dict.values())
            results_dict.append({'sentence_id': dict_values[0], 'distance': similarity_value_list[index]})
        break

    #print(results_dict)

else:
    print('!!! NO ENTITY !!! CANNOT COMPARE !!!')

[{'sentence_id': 1, 'distance': 0.0}, {'sentence_id': 2, 'distance': 0.2699116577941307}, {'sentence_id': 3, 'distance': 0.2699116577941307}, {'sentence_id': 4, 'distance': 0.24958146813922846}, {'sentence_id': 5, 'distance': 0.2698614381467951}, {'sentence_id': 6, 'distance': 0.2698614381467951}, {'sentence_id': 7, 'distance': 0.2699116577941307}, {'sentence_id': 8, 'distance': 0.2699116577941307}, {'sentence_id': 9, 'distance': 0.0}, {'sentence_id': 10, 'distance': 0.0}, {'sentence_id': 11, 'distance': 0.2699116577941307}, {'sentence_id': 12, 'distance': 0.2699116577941307}, {'sentence_id': 13, 'distance': 0.24040228144975806}, {'sentence_id': 14, 'distance': 0.2544169851074575}, {'sentence_id': 15, 'distance': 0.24397267619466598}, {'sentence_id': 16, 'distance': 0.2818472306716152}, {'sentence_id': 17, 'distance': 0.0}, {'sentence_id': 18, 'distance': 0.0}, {'sentence_id': 19, 'distance': 0.0}, {'sentence_id': 20, 'distance': 0.494336180799457}, {'sentence_id': 21, 'distance': 0.0}

In [59]:
### SBERT ###

sentences = sbert_sentences

sentence_embeddings = np.load('../datasets/exp3_3926.npy')

In [60]:
query = query_sentence

queries = [query]
query_embeddings = model.encode(queries)

for query, query_embedding in zip(queries, query_embeddings):
    distances_sbert = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]
    print(distances_sbert)   

[0.56687249 0.49426758 0.4166684  ... 0.57170349 0.63536532 0.61047704]


In [61]:
#print(distances_sbert)
#print(results_dict)


new_result_dict = []

for i in range(0, len(results_dict)):
    new_result_dict.append({'sentence_new_index': i+1, 'distance': ((results_dict[i]['distance'] + distances_sbert[i]) / 2.0)})


# print(new_result_dict)

In [62]:
### SIRALAMA ###

results_dict_2 = sorted(new_result_dict, key=lambda x: float(x['distance']), reverse=True)    
result_10 = results_dict_2[:10]

son = []
for i, _dict in enumerate(result_10):
    for index, val in enumerate(sbert_sentences):
        if index == _dict['sentence_new_index']:
            son.append({'sentence' : sbert_sentences[index], 'distance': _dict['distance']})
            
for i, val in enumerate(son):
    print(i+1, val)
    print("\n")

1 {'sentence': 'verilecek ceza , birinci fıkraya giren hallerde üç yıldan , üçüncü fıkraya giren hallerde beş yıldan az olamaz .', 'distance': 0.7406497505656895}


2 {'sentence': 'veya tehdit kullanılarak ya da hukuka aykırı başka bir davranışla bir siyasi partinin faaliyetlerinin engellenmesi halinde , iki yıldan beş yıla kadar hapis cezasına hükmolunur .', 'distance': 0.6928250458274879}


3 {'sentence': 'aleni olarak işlenmişse iki yıldan beş yıla kadar hapis cezası verilir .', 'distance': 0.6883615492977022}


4 {'sentence': 'veya tehdit kullanılarak ya da hukuka aykırı başka bir davranışla ; Bir kamu faaliyetinin yürütülmesine , Kamu kurumlarında veya kamu kurumu niteliğindeki meslek kuruluşlarında verilen ya da kamu makamlarının verdiği izne dayalı olarak sunulan hizmetlerden yararlanılmasına , engel olunması halinde , fail hakkında iki yıldan beş yıla kadar hapis cezasına hükmolunur .', 'distance': 0.6873594506148982}


5 {'sentence': 'uyarınca elkonulan ürün , cihaz ve makinel